## Dependencies

In [ ]:
!pip install weaviate-client

## Configuration

In [ ]:
import weaviate
import json

client = weaviate.Client(
  url="WEAVIATE-INSTANCE-URL",  # URL of your Weaviate instance
  auth_client_secret=weaviate.AuthApiKey(api_key="AUTH-KEY"), # (Optional) If the Weaviate instance requires authentication
)

client.is_ready()

## Schema

In [ ]:
# resetting the schema. CAUTION: This will delete your collection 
if client.schema.exists("MyCollection"):
    client.schema.delete_class("MyCollection")

schema = {
    "class": "MyCollection",
    "vectorizer": "none",
    "vectorIndexConfig": {
        "distance": "cosine" # make sure to provide the distance metric to search through your vectors
    },
}

client.schema.create_class(schema)

print("Successfully created the schema.")

## Import the Data

In [ ]:
data = [
   {
      "title": "First Object",
      "foo": 99, 
      "vector": [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
   },
   {
      "title": "Second Object",
      "foo": 77, 
      "vector": [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
   },
   {
      "title": "Third Object",
      "foo": 55, 
      "vector": [0.3, 0.1, -0.1, -0.3, -0.5, -0.7]
   },
   {
      "title": "Fourth Object",
      "foo": 33, 
      "vector": [0.4, 0.41, 0.42, 0.43, 0.44, 0.45]
   },
   {
      "title": "Fifth Object",
      "foo": 11,
      "vector": [0.5, 0.5, 0, 0, 0, 0]
   },
]

client.batch.configure(batch_size=10)  # Configure batch

# Batch import all objects
# yes batch is an overkill for 5 objects, but it is recommended for large volumes of data
with client.batch as batch:
  for item in data:

      properties = {
         "title": item["title"],
         "foo": item["foo"],
      }

      # the call that performs data insert
      client.batch.add_data_object(
         class_name="MyCollection",
         data_object=properties,
         vector=item["vector"] # your vector embeddings go here
      )

print("Data import complete")

Quick check to see if all objects are in.
Let's use [meta count](https://weaviate.io/developers/weaviate/search/aggregate#retrieve-a-meta-property).

In [ ]:
# Check number of objects
response = (
    client.query
    .aggregate("MyCollection")
    .with_meta_count()
    .do()
)

print(response)

## Query Weaviate: Vector Search (vector embeddings)

Available types of queries you can run when working with vector embeddings (without modules) in **Weaviate**:

1. [nearVector](https://weaviate.io/developers/weaviate/api/graphql/vector-search-parameters#nearvector)

2. [nearObject](https://weaviate.io/developers/weaviate/api/graphql/vector-search-parameters#nearobject)

### nearVector Example

**First example** - Search Weaviate with a vector embedding, and return title property.

See [the docs](https://weaviate.io/developers/weaviate/search/similarity#a-vector) for more.

In [ ]:
response = (
    client.query
    .get("MyCollection", ["title"])
    .with_near_vector({
        "vector": [-0.012, 0.021, -0.23, -0.42, 0.5, 0.5]
    })
    .with_limit(2) # limit the output to only 2
    .do()
)

result = response["data"]["Get"]["MyCollection"]
print(json.dumps(result, indent=2))

**Second example** - The same search query, but this time also return `distance`, `vector`, and `id`.

In [ ]:
import json
response = (
    client.query
    .get("MyCollection", ["title"])
    .with_near_vector({
        "vector": [-0.012, 0.021, -0.23, -0.42, 0.5, 0.5]
    })
    .with_limit(2) # limit the output to only 2
    .with_additional(["distance", "vector, id"]) # output the distance of the query vector to the objects in the database
    .do()
)

result = response["data"]["Get"]["MyCollection"]
print(json.dumps(result, indent=2))

**Third example** – Same vector query, but this time we will filter on "foo" (which should be greater than 44). Also, let's return "title" and "foo".

See [the docs](https://weaviate.io/developers/weaviate/search/filters#a-single-condition-filter) for more.

In [ ]:
import json
response = (
    client.query
    .get("MyCollection", ["title", "foo"])
    .with_near_vector({
        "vector": [-0.012, 0.021, -0.23, -0.42, 0.5, 0.5]
    })
    .with_additional(["distance, id"]) # output the distance of the query vector to the objects in the database
    .with_where({
        "path": ["foo"],
        "operator": "GreaterThan",
        "valueNumber": 44
    })
    .with_limit(2) # limit the output to only 2
    .do()
)

result = response["data"]["Get"]["MyCollection"]
print(json.dumps(result, indent=2))

### nearObject Example

Weaviate also allows you to search for similar objects.

See [the docs](https://weaviate.io/developers/weaviate/search/similarity#an-object) for more.

**Fourth example** - 
Search through `MyCollection` for similar objects, by providing an id from the previous query. 

> Note #1: The id was taken from the query above <br/>
> The generated id for you might be different.

> Note #2: The first object returned is always itself.

In [ ]:
response = (
    client.query
    .get("MyCollection", ["title"])
    .with_near_object({ # the id of the the search object
        "id": "a9866375-03f7-491a-9a14-5152efe2ea09"
    })
    .with_limit(3)
    .with_additional(["distance"])
    .do()
)

result = response["data"]["Get"]["MyCollection"]
print(json.dumps(result, indent=2))